# VGG
---
VGG（Visual Geometry Group）网络是由牛津大学的Visual Geometry Group团队在2014年提出的一种深度卷积神经网络架构。它在当年的ImageNet图像分类挑战赛中取得了优异的成绩，并因其简洁的结构和强大的性能而广受欢迎。

## 主要特点
1. *深度：*VGG网络以其深度著称，最深的版本（VGG19）有19层（包括卷积层和全连接层），而VGG16有16层。这种深度在当时是非常先进的。

2. *小卷积核：*VGG网络主要使用3x3的小卷积核，而不是像AlexNet那样使用较大的卷积核（如11x11）。使用多个3x3卷积核的堆叠可以模拟更大感受野的效果，同时减少参数数量。

3. *固定结构：*VGG网络的结构非常规整，每层卷积层后面都跟着一个ReLU激活函数，然后是最大池化层。这种固定的结构使得网络的设计和实现非常简单。

4. *全连接层：*VGG网络在卷积层之后使用了三个全连接层，最后是一个softmax层用于分类。全连接层的参数数量非常多，这也是VGG网络的一个缺点。

## 网络结构

VGG网络有多个版本，最常见的是VGG16和VGG19。以下是VGG16的结构：

1. *卷积层：*

    - 前两个卷积块（Block）各有2个卷积层，每个卷积层使用3x3的卷积核，步幅为1，填充为1。

    - 后三个卷积块各有3个卷积层，每个卷积层同样使用3x3的卷积核，步幅为1，填充为1。

    - 每个卷积层后面都跟着一个ReLU激活函数。

2. 池化层：

    - 每个卷积块后面都有一个最大池化层，使用2x2的池化窗口，步幅为2。

3. 全连接层：

    - 经过5个卷积块后，特征图被展平成一个向量，然后通过三个全连接层。

    - 前两个全连接层各有4096个神经元，最后一个全连接层有1000个神经元（对应ImageNet的1000个类别）。

    - 每个全连接层后面都跟着一个ReLU激活函数和一个Dropout层（Dropout率为0.5）。

4. 输出层：

    - 最后一个全连接层后面是一个softmax层，用于输出分类概率。

![VGG](https://zh-v2.d2l.ai/_images/vgg.svg "VGG")


## 优点
    简洁性：VGG网络的结构非常简洁，易于理解和实现。

    深度：通过增加网络深度，VGG网络能够学习到更复杂的特征。

    小卷积核：使用多个3x3卷积核的堆叠可以模拟更大感受野的效果，同时减少参数数量。

## 缺点
    参数数量多：尽管卷积层的参数数量相对较少，但全连接层的参数数量非常多，导致整个网络的参数量非常大，训练和推理的计算成本较高。

    内存消耗大：由于网络深度较大，训练时需要大量的内存来存储中间特征图。





下面的代码实现了VGG-11。可以通过在conv_arch上执行for循环来简单实现。

In [7]:
import torch
from torch import nn
from d2l import torch as d2l


def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

In [8]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [9]:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        # 全连接层部分
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

接下来，我们将构建一个高度和宽度为224的单通道数据样本，以观察每个层输出的形状。

In [10]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])
